In [48]:
import pandas as pd
import ast
import os,sys
import subprocess
from datetime import datetime
import backtrader as bt

os.chdir('/Volumes/Data/Chipy/Py-Finance/Crypto/TechnicalAnalysis/')

In [24]:
class SmaCross(bt.SignalStrategy):
    params = (('pfast', 10), ('pslow', 30),('printlog',False))
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        sma1, sma2 = bt.ind.SMA(period=self.p.pfast), bt.ind.SMA(period=self.p.pslow)
        self.signal_add(bt.SIGNAL_LONG, bt.ind.CrossOver(sma1, sma2))

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def stop(self):
        test_run = {}
        test_run['pfast'] = self.params.pfast
        test_run['pslow'] = self.params.pslow
        test_run['final_value'] = (self.broker.getvalue()+self.broker.getcash())
        self.log('|{}'.format(test_run),doprint=True)
        #self.log('(pfast %2d) (pslow %2d) Ending Value %.2f' % (self.params.pfast,self.params.pslow,self.broker.getvalue()), doprint=True)

In [28]:
cerebro = bt.Cerebro()

#data = bt.feeds.YahooFinanceData(dataname='YHOO', fromdate=datetime(2011, 1, 1),todate=datetime(2012, 12, 31))
data = bt.feeds.GenericCSVData(dataname='btc_usd.csv',dtformat='%Y-%m-%d',openinterest=-1)

cerebro.adddata(data)
cerebro.addsizer(bt.sizers.PercentSizer,percents=100)

#cerebro.addstrategy(SmaCross)
strats = cerebro.optstrategy(SmaCross,pfast=range(5,12),pslow=range(15,35))

cerebro.addsizer(bt.sizers.PercentSizer,percents=100)

f = open('output.txt','w')
cerebro.run()
sys.stdout = f

In [38]:
filename = 'output.txt'
with open(filename) as f:
    content = f.readlines()
content = [x.split('|')[1].strip("\n").replace('"', '') for x in content] 

In [50]:
run_data = pd.DataFrame([ast.literal_eval(x) for x in content])
run_data.head()

,final_value,pfast,pslow
0,126377.459873,8,15
1,50471.596010,5,16
2,161793.475937,7,16
3,43316.287646,10,16
4,69340.387397,7,17


In [51]:
run_data.sort_values('final_value',ascending=False).head(10)

,final_value,pfast,pslow
42,251363.541447,6,19
8,229456.843850,9,19
9,218680.002114,6,20
34,199189.209935,8,34
67,187988.106173,8,33
35,174245.468841,7,15
11,163047.507232,7,21
2,161793.475937,7,16
56,139395.713247,6,27
47,133606.429424,5,22


In [ ]:
#Beware overfitting!